## Single-Server Queue

### Direct Event Scheduling

In [3]:
from collections import deque
import random, simulus

# set the global random seed for repeatability
random.seed(13579)

MEAN_IATIME = 1.0 # mean inter-arrival time
MEAN_STIME = 0.8 # mean service time

def iatime(rng): 
    return rng.expovariate(1/MEAN_IATIME)

def svtime(rng):
    return rng.expovariate(1/MEAN_STIME)

# all jobs in system are stored here
queue = deque()

def arrive(sim):
    queue.append((num_arrivals, sim.now))
    print('%g: job[%d] arrives (jobs_in_system=%d)' % 
          (sim.now, num_arrivals, len(queue)))

    # schedule next job's arrival
    sim.sched(arrive, sim, offset=iatime(sim.rng()))
    
    if len(queue) == 1:
        # no one was in service, there's no need to wait
        print('%g: job[%d] enters service (queue_time=0)' % 
              (sim.now, queue[0][0]))
        
        # schedule the job's departure
        sim.sched(depart, sim, offset=svtime(sim.rng()))
        
def depart(sim):
    j, t = queue.popleft()
    print('%g: job[%d] departs (time_in_system=%g, jobs_in_system=%d)' % 
          (sim.now, j, sim.now-t, len(queue)))
    
    if len(queue) > 0:
        # there is waiting job, serve the next one
        print('%g: job[%d] enters service (queue_time=%g)' % 
              (sim.now, queue[0][0], sim.now-queue[0][1]))
        sim.sched(depart, sim, offset=svtime(sim.rng()))
        
sim = simulus.simulator('ssq')
sim.sched(arrive, sim, offset=iatime(sim.rng()))
sim.run(10)

NameError: name 'num_arrivals' is not defined

### Processes and Resource

In [ ]:
import random, simulus

# set the global random seed for repeatability
random.seed(13579)

MEAN_IATIME = 1.0 # mean inter-arrival time
MEAN_STIME = 0.8 # mean service time

def iatime(rng): 
    return rng.expovariate(1/MEAN_IATIME)

def svtime(rng):
    return rng.expovariate(1/MEAN_STIME)

def job(sim, job_id, rsc):
    t = sim.now
    print('%g: job[%d] arrives (jobs_in_system=%d)' % 
          (sim.now, job_id, rsc.num_in_system()+1))
    
    rsc.acquire()
    print('%g: job[%d] enters service (queue_time=%g)' % 
          (sim.now, job_id, sim.now-t))
    
    sim.sleep(svtime(sim.rng()))
    rsc.release()
    print('%g: job[%d] departs (time_in_system=%g, jobs_in_system=%d)' % 
          (sim.now, job_id, sim.now-t, rsc.num_in_system()))
    
def arrive(sim, rsc):
    job_id = 0
    while True:
        sim.sleep(iatime(sim.rng()))
        sim.process(job, sim, job_id, rsc)
        job_id += 1

sim = sm.simulator('ssq')
rsc = sim.resource()
sim.process(arrive, sim, rsc)
sim.run(10)